In [4]:
# Autor: Adaptación basada en D-Wave
# Uso: Envío de instancias cúbicas convertidas a QUBO al backend Dirac-1 de QCI

import ast
import dimod
import numpy as np
import time
import os
from dimod import Vartype
import qci_client as qc

# ---------- Funciones Auxiliares ---------- #

def TTS(p, t):
    """Calcula el Time-To-Solution seguro, evitando divisiones por cero."""
    if p == 0:
        return np.inf
    eps = 1e-10
    p = np.clip(p, eps, 1 - eps)
    return t * np.max([1.0, np.abs(np.log(0.01) / np.log(1 - p))])

def parse_problems(_problems):
    """Convierte la estructura de problema en diccionarios de términos lineales, cuadráticos y cúbicos."""
    linear = _problems[0][0]
    linear.update(_problems[0][1])
    quadratic = _problems[1][0]
    quadratic.update(_problems[1][1])
    quadratic.update(_problems[1][2])
    cubic = _problems[2][0]
    return linear, quadratic, cubic

def eval_cubic_energies(_linear, _quadratic, _cubic, _samples):
    _samples = np.atleast_2d(_samples)
    linmat = np.zeros((1, _samples.shape[1]), dtype=int)
    quadmat = np.zeros((3, len(_quadratic)), dtype=int)
    cubmat = np.zeros((4, len(_cubic)), dtype=int)
    
    linmat[0, :] = np.array([_linear[key] for key in range(len(_linear))])
    quadmat[0:2, :] = np.asarray(list(_quadratic.keys())).T
    quadmat[2, :] = np.array(list(_quadratic.values()))
    cubmat[0:3, :] = np.asarray(list(_cubic.keys())).T
    cubmat[3, :] = np.array(list(_cubic.values()))
    
    ret = np.sum(_samples * linmat, axis=1)
    temp = _samples[:, quadmat[0, :]] * _samples[:, quadmat[1, :]] * quadmat[2, :]
    ret += np.sum(temp, axis=1)
    temp = _samples[:, cubmat[0, :]] * _samples[:, cubmat[1, :]] * _samples[:, cubmat[2, :]] * cubmat[3, :]
    ret += np.sum(temp, axis=1)
    
    return np.asarray(ret)


def make_ising_instance(_linear, _quadratic, _cubic):
    """Convierte un problema cúbico a BQM (modelo ising SPIN) usando gadgets."""
    bqm = dimod.BinaryQuadraticModel(vartype=dimod.SPIN)
    for v in sorted(_linear):
        bqm.add_variable(v)
        bqm.add_linear(v, _linear[v])
    for (u, v), bias in _quadratic.items():
        bqm.add_quadratic(u, v, bias)

    for (x, y, z), sign in _cubic.items():
        u = len(bqm)
        v = u + 1
        bqm.add_variable(u)
        bqm.add_variable(v)

        if np.abs(y - z) > 2:  # Gadget vertical
            if options['BETTER_GADGETS']:
                bqm.add_linear(u, 6)
                bqm.add_linear(x, -3)
                bqm.add_linear(y, -3)
                bqm.add_linear(z, sign)
                bqm.add_quadratic(u, x, -4)
                bqm.add_quadratic(u, y, -4)
                bqm.add_quadratic(u, z, 2 * sign)
                bqm.add_quadratic(v, y, -1)
                bqm.add_quadratic(v, z, -sign)
                bqm.add_quadratic(x, y, 1)
                bqm.add_quadratic(x, z, -sign)
            else:
                bqm.add_linear(z, sign)
                bqm.add_quadratic(v, y, -1)
                bqm.add_quadratic(v, z, -sign)
                bqm.add_quadratic(u, x, -4)
                bqm.add_quadratic(u, y, -4)
                bqm.add_quadratic(u, z, 2 * sign)
                bqm.add_quadratic(x, z, -sign)
                if sign == 1:
                    bqm.add_linear(u, 6)
                    bqm.add_linear(x, -3)
                    bqm.add_linear(y, -3)
                    bqm.add_quadratic(x, y, 1)
                else:
                    bqm.add_linear(u, 2)
                    bqm.add_linear(x, -1)
                    bqm.add_linear(y, -1)
                    bqm.add_quadratic(x, y, 3)
        else:  # Gadget horizontal
            bqm.add_linear(u, -2)
            bqm.add_linear(v, -1)
            bqm.add_linear(x, -1)
            bqm.add_linear(y, -1)
            bqm.add_quadratic(u, x, 2)
            bqm.add_quadratic(u, y, 2)
            bqm.add_quadratic(v, x, 1)
            bqm.add_quadratic(v, y, 1)
            bqm.add_quadratic(v, z, sign)
            bqm.add_quadratic(u, v, 2)
            bqm.add_quadratic(x, y, 1)

    return bqm

def bqm_to_qubo_matrix(bqm_bin):
    """Convierte un BQM binario en matriz QUBO (numpy)."""
    n = len(bqm_bin.linear)
    Q = np.zeros((n, n))
    for v, bias in bqm_bin.linear.items():
        Q[v, v] = bias
    for (u, v), bias in bqm_bin.quadratic.items():
        Q[u, v] += bias /2 
        Q[v, u] += bias /2 # redundante, simétrica
    return Q

# ---------- Configuración ---------- #
options = {
    "RUN_ALL": False,
    "BETTER_GADGETS": False,
    "NUM_REPETITIONS": 3
}

# Sampler local clásico
sampler = dimod.SimulatedAnnealingSampler()

# Lectura de soluciones óptimas
with open("optimal_solutions/ibm_washington_cubic_instances_minimum_energies.txt") as f:
    gse = ast.literal_eval(f.read())
with open("optimal_solutions/ibm_washington_cubic_instances_maximum_energies.txt") as f:
    maxene = ast.literal_eval(f.read())

data = []
seed = 5
tic = time.time()

# ---------- Carga y envío de problema ---------- #

# 1) Leer la instancia cúbica
with open(f"problem_instances/ibm_washington_{seed}.txt") as f:
    problems = ast.literal_eval(f.read())
linear, quadratic, cubic = parse_problems(problems)

# 2) Convertir a modelo Ising con gadgets
bqm_spin = make_ising_instance(linear, quadratic, cubic)
print("Número total variables en BQM (SPIN + gadgets):", len(bqm_spin))

# 3) Convertir a QUBO binario
bqm_binary = bqm_spin.change_vartype(Vartype.BINARY, inplace=False)
qubo_matrix = bqm_to_qubo_matrix(bqm_binary)

# 4) Enviar QUBO a Dirac-1
os.environ['QCI_API_URL'] = 'https://api.qci-prod.com'
os.environ['QCI_TOKEN'] = 'TOKEN'
client = qc.QciClient()

file_def = {
    "file_name": f"qubo_seed_{seed}",
    "file_config": {"qubo": {"data": qubo_matrix}}
}
file_id = client.upload_file(file=file_def)["file_id"]

job_body = client.build_job_body(
    job_type="sample-qubo",
    qubo_file_id=file_id,
    job_params={"device_type": "dirac-1", "num_samples": 100}
)
job_id = client.process_job(job_body=job_body)["job_info"]["job_id"]

# 5) Esperar resultados
status = client.get_job_status(job_id=job_id)
while status["status"] not in ["COMPLETED", "FAILED"]:
    time.sleep(5)
    status = client.get_job_status(job_id=job_id)

results = client.get_job_results(job_id=job_id)
solutions = results["results"]["solutions"]

print(results)

wall_time = time.time() - tic

# 6) Convertir a SPIN y quedarnos con variables originales
spin_solutions = 2 * np.array(solutions) - 1
spin_original = spin_solutions[:, :len(linear)]

# 7) Evaluar energía cúbica pura
energies = eval_cubic_energies(linear, quadratic, cubic, spin_original)
min_energy = np.min(energies)
mean_energy = np.mean(energies)

# 8) Métricas de evaluación
prob_gs = np.sum(energies == gse[seed]) / len(energies)
ar = 1 - (mean_energy - gse[seed]) / (maxene[seed] - gse[seed])
sample_time = wall_time / len(solutions)
tts_val = TTS(prob_gs, sample_time)

# 9) Guardar resultados
data.append([
    seed, mean_energy, min_energy, gse[seed],
    prob_gs, wall_time, wall_time, tts_val,
    sample_time, None, None
])

print(f"\nSEED {seed}: Mean energy {mean_energy:.4f}, min energy {min_energy}, optimal value {gse[seed]},"
      f"PGS={prob_gs:.6f}, AR={ar:.4f}")
print(f"     wall time: {wall_time:.3f}s, sample time: {sample_time:.6f}s, TTS: {tts_val:.6f}s")


Número total variables en BQM (SPIN + gadgets): 265
2025-06-18 23:21:45 - Dirac allocation balance = 577 s
2025-06-18 23:21:45 - Job submitted: job_id='6853ecdeb8b42e8edbee8509'
2025-06-18 23:21:45 - QUEUED
2025-06-18 23:21:48 - RUNNING
2025-06-19 13:37:44 - COMPLETED
2025-06-19 13:37:46 - Dirac allocation balance = 315 s
{'job_info': {'job_id': '6853ecdeb8b42e8edbee8509', 'job_submission': {'problem_config': {'quadratic_unconstrained_binary_optimization': {'qubo_file_id': '6853ecdd5e085526322a4555'}}, 'device_config': {'dirac-1': {'num_samples': 100}}}, 'job_status': {'submitted_at_rfc3339nano': '2025-06-19T10:56:30.669Z', 'queued_at_rfc3339nano': '2025-06-19T10:56:30.67Z', 'running_at_rfc3339nano': '2025-06-19T10:56:31.676Z', 'completed_at_rfc3339nano': '2025-06-19T11:37:43.425Z'}, 'job_result': {'file_id': '6853f6875e085526322a4557', 'device_usage_s': 262}}, 'status': 'COMPLETED', 'results': {'counts': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [5]:
print(spin_solutions)

[[ 1  1  1 ...  1  1  1]
 [ 1  1  1 ...  1  1  1]
 [ 1  1  1 ... -1 -1  1]
 ...
 [-1  1 -1 ...  1  1  1]
 [ 1 -1  1 ...  1  1  1]
 [-1 -1  1 ...  1  1 -1]]


In [6]:
def descend_to_local_minimum(samples, linear, quadratic, cubic, max_iters=100):
    samples = samples.copy()
    n = samples.shape[1]

    for i in range(samples.shape[0]):  # Para cada muestra
        for _ in range(max_iters):
            improved = False
            for j in range(n):  # Para cada variable
                flip = samples[i].copy()
                flip[j] *= -1

                e0 = eval_cubic_energies(linear, quadratic, cubic, [samples[i]])[0]
                e1 = eval_cubic_energies(linear, quadratic, cubic, [flip])[0]

                if e1 < e0:
                    samples[i] = flip
                    improved = True
                    break  # Reinicia búsqueda
            if not improved:
                break  # Ya es mínimo local
    return samples


In [7]:
# 7) Evaluar energía cúbica pura (pre-postprocesado)
energies = eval_cubic_energies(linear, quadratic, cubic, spin_original)

# 8) Descenso local (postprocesamiento)
spin_post = descend_to_local_minimum(spin_original, linear, quadratic, cubic)
pp_energies = eval_cubic_energies(linear, quadratic, cubic, spin_post)

# 9) Métricas de evaluación
min_energy = np.min(energies)
mean_energy = np.mean(energies)
mean_pp_energy = np.mean(pp_energies)
min_pp_energy = np.min(pp_energies)

prob_gs = np.sum(energies == gse[seed]) / len(energies)
prob_pp_gs = np.sum(pp_energies == gse[seed]) / len(pp_energies)

ar = 1 - (mean_energy - gse[seed]) / (maxene[seed] - gse[seed])
ar_pp = 1 - (mean_pp_energy - gse[seed]) / (maxene[seed] - gse[seed])

sample_time = wall_time / len(solutions)
tts_val = TTS(prob_gs, sample_time)
tts_pp_val = TTS(prob_pp_gs, sample_time)

# 10) Guardar resultados
data.append([
    seed,
    mean_energy,
    min_energy,
    gse[seed],
    prob_gs,
    wall_time,
    wall_time,
    tts_val,
    sample_time,
    prob_pp_gs,
    tts_pp_val
])

# 11) Imprimir resultados
print(f"\nSEED {seed}:")
print(f"   Mean energy       (pre): {mean_energy:.4f}, min: {min_energy}, AR={ar:.4f}, PGS={prob_gs:.6f}")
print(f"   Mean energy (postproc): {mean_pp_energy:.4f}, min: {min_pp_energy}, AR={ar_pp:.4f}, PGS={prob_pp_gs:.6f}")
print(f"   wall time: {wall_time:.3f}s, sample time: {sample_time:.6f}s")
print(f"   TTS (pre): {tts_val:.6f}s, TTS (post): {tts_pp_val:.6f}s")



SEED 5:
   Mean energy       (pre): -176.4000, min: -192, AR=0.9446, PGS=0.000000
   Mean energy (postproc): -185.4400, min: -196, AR=0.9678, PGS=0.000000
   wall time: 51367.074s, sample time: 513.670743s
   TTS (pre): infs, TTS (post): infs
